In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df_train = pd.read_csv('INPUT/train.csv')
df_train.head()

In [ ]:
im = plt.imread('INPUT/traind/train_1.jpg')

In [ ]:
plt.imshow(im)

In [ ]:
len(df_train)

In [ ]:
labels = df_train['tags'].apply(lambda x: x.split(' '))

In [ ]:
len(labels)

In [ ]:
labels[0]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
s = mlb.fit_transform(labels)

In [ ]:
s

In [ ]:
s.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Lambda, Flatten, Dense
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Dropout
from keras import backend as K

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

In [ ]:
import cv2
#df_train = df_train[:5000]
#s = s[:5000]

images = []
for img in df_train['image_name']:
    name = 'INPUT/traind/' + img +'.jpg'
    image_arr = cv2.imread(name)
    image_arr = cv2.resize(image_arr, (50, 50), interpolation=cv2.INTER_CUBIC)
    images.append(image_arr)
images = np.array(images)
labels = np.array(s)

In [ ]:
def getModel():
    model = Sequential()
    # Preprocess incoming data, centered around zero with small standard deviation 
    model.add(Lambda(lambda x: x/127.5 - 1.,
                    input_shape=(50, 50, 3)))
                    #output_shape=(row, col, ch)))

    model.add(Conv2D(16, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(16, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(24, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(24, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(17, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid= train_test_split(images, labels, test_size=0.25, random_state=42)

In [ ]:
model = getModel()
hist = model.fit(X_train, y_train, epochs=12, verbose=2, validation_data=(X_valid, y_valid))

In [ ]:
df_sample = pd.read_csv("INPUT/sample_submission.csv")
df_sample.head()

In [ ]:
df_sample.tail()

In [ ]:
images_ = []
for img in df_sample['image_name']:
    #print(img)
    name = 'INPUT/testd/' + img +'.jpg'
    image_arr = cv2.imread(name)
    image_arr = cv2.resize(image_arr, (50, 50), interpolation=cv2.INTER_CUBIC)
    images_.append(image_arr)
images_test = np.array(images_)
#images_test = images_test[:10000]
#labels = np.array(s)

In [ ]:
len(images_test)

In [ ]:
result = model.predict(images_test)

In [ ]:
result[result>=0.3] = 1
result[result<0.3] = 0

In [ ]:
result[0]

In [ ]:
act_result = mlb.inverse_transform(result)

In [ ]:
df_sample['tags'] = [" ".join(lab) for lab in act_result]

In [ ]:
df_sample.to_csv("out.csv", index=False)

In [ ]:
check = pd.read_csv("out.csv")

In [ ]:
check.head()

In [ ]:
check[10000:11000]

import sklearn
from sklearn.utils import shuffle

import cv2
def generator(samples, labels, batch_size=64):
    num_samples = len(samples)
    while 1:
        #shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            label_samples = labels[offset:offset+batch_size]
            images = []
            labels = []
            for (image, label) in zip(batch_samples, label_samples):
                name = '../input/train-jpg/' + image +'.jpg'
                #print(name)
                image_arr = cv2.imread(name)
                image_arr = cv2.resize(image_arr, (50, 50), interpolation=cv2.INTER_CUBIC)
                images.append(image_arr)
                labels.append(label)

            X_train = np.array(images)
            y_train = np.array(labels)
            yield sklearn.utils.shuffle(X_train, y_train)
train_generator = generator(df_train['image_name'], s, batch_size=64)
#validation_generator = generator(validation_samples, batch_size=64)